In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# IMPORT LIBRARIES

In [ ]:
import keras
import keras_nlp

# LOAD GEMMA MODEL

In [ ]:
"""
# #Download the model into your working directory 
# import kagglehub

# # Download latest version
# path = kagglehub.model_download("google/gemma/transformers/2b-it")

# print("Path to model files:", path)

"""


In [ ]:
%%time
# Load the gemma model via Keras
LLM = keras_nlp.models.GemmaCausalLM.from_preset('gemma_instruct_2b_en')

# DATA WRANGLING

In [ ]:
# read the scraped reviews dataset
review_df = pd.read_csv("/kaggle/input/restaurant-review-data/combined_restaurant_review_df.csv")

In [ ]:
print(review_df.shape)
review_df.head(2)

In [ ]:
# Function to select restaurants and comments
def select_restaurants_and_reviews(df):
    print("Before drop",df.shape)
    # Step1: Do the grouping
    review_counts = df.groupby(['Location', 'restaurant_name']).size().reset_index(name='review_count')
    # Step 2: Filter restaurants with at least 25 reviews
    filtered_df = review_counts[review_counts['review_count'] >= 25]
    # Step 3: Merge back with the original DataFrame to get full details of those restaurants
    valid_restaurants = pd.merge(df, filtered_df[['Location', 'restaurant_name']], on=['Location', 'restaurant_name'])
    print("After drop",valid_restaurants.shape)
    # Step4: Sort comments by rating in ascending order and select the first 5 comments per restaurant
    rating_sorted = valid_restaurants.groupby(['Location', 'restaurant_name']).apply(lambda x: x.sort_values(by='restaurant_rating').head(5)).reset_index(drop=True)
    # We can select the top 3 and the last 2 reviews per restaurant instead of grabbing the first 5 reviews
#    rating_sorted = valid_restaurants.groupby(['Location', 'restaurant_name']).apply(
#         lambda x: pd.concat([
#             x.sort_values(by='restaurant_rating', ascending=False).head(3),  # Top 3 
#             x.sort_values(by='restaurant_rating', ascending=False).tail(2)    # Last 2
#         ])
#     ).reset_index(drop=True)
    return rating_sorted

In [ ]:
# Call the function and display results
selected_reviews = select_restaurants_and_reviews(review_df)
selected_reviews.shape

In [ ]:
selected_reviews.groupby(['Location']).size().head()

In [ ]:
review_counts = selected_reviews.groupby([ 'Location','restaurant_name']).size().reset_index(name='review_count')
review_counts.tail()

# SAVE THE OUTPUT

In [ ]:
selected_reviews.to_csv("selected_reviews.csv", index = False)
review_counts.to_csv("selected_reviews_count.csv", index = False)

In [ ]:
print("All done")